In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import plotly.graph_objects as go

## Lecture des datas

In [3]:
df = pd.read_csv("Data/all_data.csv", sep=";")
print(f"{len(df)} tickers")

1247 tickers


## Exclusions

### Tabac / Armes controversées

In [4]:
df2 = df.loc[(df['CWEAP_TIE']!=1) & (df['TOB_PRODUCER']!=1),:]
print(f"{len(df2)} tickers")

1237 tickers


Elimination de 10 entreprises

### Controverses 0/1

In [5]:
columns_controverse = ["E_CONTROVERSY", "S_CONTROVERSY", 'G_CONTROVERSY']

In [6]:
# Controverse E
for col in columns_controverse:
    print(f"NB {col} : {len(df.loc[df[col]<2,:])}")

NB E_CONTROVERSY : 28
NB S_CONTROVERSY : 149
NB G_CONTROVERSY : 35


In [7]:
df3 = df2.loc[~df2[columns_controverse].lt(2).any(axis=1), :]
print(f"Elimination {len(df2)-len(df3)}")
print(f"Reste {len(df3)}")

Elimination 175
Reste 1062


### Note ESG < BB eet E > 3

In [8]:
print(f"NB : {len(df.loc[df["IVA_COMPANY_RATING"].isin(['B','CCC']),:])}")
df4 = df3.loc[~df3["IVA_COMPANY_RATING"].isin(['B','CCC']),:]
print(f"Elimination {len(df3)-len(df4)}")
print(f"Reste {len(df4)}")

NB : 17
Elimination 11
Reste 1051


In [9]:
print(f"NB : {len(df.loc[df["E_SCORE"]<=3,:])}")
df5 = df4.loc[df4["E_SCORE"]>3,:]
print(f"Elimination {len(df4)-len(df5)}")
print(f"Reste {len(df5)}")

NB : 64
Elimination 58
Reste 993


### Restrictions ODD

In [10]:
# odds = ["07","12","13"]
odds = [x for x in df.columns if x.startswith("SDG")]
for odd in odds:
    print(f"NB {odd} : {len(df.loc[df[odd]<=-2,:])}")

df6 = df5.loc[~df5[odds].le(-2).any(axis=1), :]
print(f"Elimination {len(df5)-len(df6)}")
print(f"Reste {len(df6)}")

df7 = df6.loc[~df6[odds].le(2).all(axis=1), :]
print(f"Elimination {len(df6)-len(df7)}")
print(f"Reste {len(df7)}")

NB SDG_01_NET_ALIGNMENT_SCORE : 28
NB SDG_02_NET_ALIGNMENT_SCORE : 7
NB SDG_03_NET_ALIGNMENT_SCORE : 48
NB SDG_04_NET_ALIGNMENT_SCORE : 16
NB SDG_05_NET_ALIGNMENT_SCORE : 1
NB SDG_06_NET_ALIGNMENT_SCORE : 14
NB SDG_07_NET_ALIGNMENT_SCORE : 83
NB SDG_08_NET_ALIGNMENT_SCORE : 6
NB SDG_09_NET_ALIGNMENT_SCORE : 37
NB SDG_10_NET_ALIGNMENT_SCORE : 9
NB SDG_11_NET_ALIGNMENT_SCORE : 51
NB SDG_12_NET_ALIGNMENT_SCORE : 106
NB SDG_13_NET_ALIGNMENT_SCORE : 83
NB SDG_14_NET_ALIGNMENT_SCORE : 64
NB SDG_15_NET_ALIGNMENT_SCORE : 66
NB SDG_16_NET_ALIGNMENT_SCORE : 28
NB SDG_17_NET_ALIGNMENT_SCORE : 70
Elimination 103
Reste 890
Elimination 157
Reste 733


## Optimisation sous contrainte du portefeuille

In [11]:
odds_sociaux = ["01","02","03","04","05","06","08","10","16"]
odds_env = ["07","11","12","13","14","15"]

df7['DURABLE_SOCIAL'] = df7[[f'SDG_{col}_NET_ALIGNMENT_SCORE' for col in odds_sociaux]].max(axis=1)
df7['DURABLE_ENVIRONNEMENTAL'] = df7[[f'SDG_{col}_NET_ALIGNMENT_SCORE' for col in odds_env]].max(axis=1)

C:\Users\bapdu\AppData\Local\Temp\ipykernel_26056\2025602295.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7['DURABLE_SOCIAL'] = df7[[f'SDG_{col}_NET_ALIGNMENT_SCORE' for col in odds_sociaux]].max(axis=1)
C:\Users\bapdu\AppData\Local\Temp\ipykernel_26056\2025602295.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7['DURABLE_ENVIRONNEMENTAL'] = df7[[f'SDG_{col}_NET_ALIGNMENT_SCORE' for col in odds_env]].max(axis=1)


In [27]:
df_fin = df7.copy()
df_fin['EST_EU_TAXONOMY_MAX_REV'] = df_fin['EST_EU_TAXONOMY_MAX_REV'].fillna(0)
df_fin['Annualized return 10Y'] = df_fin['Annualized return 10Y'].fillna(0)
# df6 = df5.loc[(df5['EST_EU_TAXONOMY_MAX_REV']>0) & (df5['ITR']>0),:] #Si Pas Taxo >0 trop de lignes => trop long a faire tourner

df_fin = df_fin.loc[(df_fin['ITR']>0),:]
# Nombre de tickers
n = len(df_fin)
print(n)
perf_moy = df_fin['Annualized return 20Y'].values
taxonomie = df_fin['EST_EU_TAXONOMY_MAX_REV'].values
itr = df_fin['ITR'].values
durable_social = df_fin['DURABLE_SOCIAL'].values
durable_env = df_fin['DURABLE_ENVIRONNEMENTAL'].values

secteurs_uniques = df_fin['gics_sector_name'].unique()  # Liste des secteurs uniques
m = len(secteurs_uniques)  # Nombre de secteurs

# Matrice binaire (1 si l'actif appartient au secteur, 0 sinon)
matrice_secteurs = np.array([[1 if df_fin.iloc[i]['gics_sector_name'] == secteur else 0 for secteur in secteurs_uniques] for i in range(n)])

# Fonction objectif (on minimise donc on prend -Perf_moy)
def objectif(w):
    return -np.dot(w, perf_moy)  # On minimise la valeur négative pour maximiser

# Contrainte : Taxonomie pondérée ≥ 10
def contrainte_taxonomie(w):
    return np.dot(w, taxonomie)-12

# Contrainte : ITR < 2
def contrainte_itr(w):
    return 1.95-np.dot(w, itr)

# Contrainte : Alignement environnemental pondéré ≥ 90%
def contrainte_alignement_social(w):
    filtres_env = (durable_env >= 2).astype(int)  # 1 si durable > 2, sinon 0

    # Calcul de la somme des poids des entreprises ayant un score durable > 2
    return np.dot(w, filtres_env) - 0.33  # Doit être ≥ 0

def contrainte_alignement_env(w):
    filtres_social = (durable_social >= 2).astype(int)  # 1 si durable > 2, sinon 0

    # Calcul de la somme des poids des entreprises ayant un score durable > 2
    return np.dot(w, filtres_social) - 0.33  # Doit être ≥ 0

def contrainte_secteur(w):
    return 0.30 - np.dot(matrice_secteurs.T, w)  # Doit être ≥ 0 pour chaque secteur

# Contrainte : Somme des poids = 1
def contrainte_somme(w):
    return np.sum(w) - 1


# Définition des contraintes sous forme de dictionnaires
contraintes = [
    {'type': 'eq', 'fun': contrainte_somme},  # Somme des poids = 1
    {'type': 'ineq', 'fun': contrainte_taxonomie},  # Taxonomie pondérée ≥ 10
    {'type': 'ineq', 'fun': contrainte_itr},  # ITR < 2
    {'type': 'ineq', 'fun': contrainte_alignement_social},  # Investissement durable Social ≥ 30%
    {'type': 'ineq', 'fun': contrainte_alignement_env}  # Investissement durable env≥ 30%
]

contraintes_secteurs = [{'type': 'ineq', 'fun': lambda w, j=j: contrainte_secteur(w)[j]} for j in range(m)]

contraintes = contraintes + contraintes_secteurs

# Bornes (chaque poids doit être positif et max 0.1)
bornes = [(0, 0.05) for _ in range(n)]

# Initialisation des poids (égalité répartie)
# w0 = np.ones(n) / n
w0 = df_fin['Weight in MSCI World'].values/df_fin['Weight in MSCI World'].values.sum()
# Résolution avec SLSQP
resultat = minimize(objectif, w0, method='SLSQP', bounds=bornes, constraints=contraintes)

# Vérification des résultats
if resultat.success:
    w_opt = resultat.x
    df_fin['Poids'] = w_opt
else:
    print("L'optimisation a échoué :", resultat.message)

731


## Résultats

### Resultats Portefeuille Article 9

In [29]:
print(f"Return PTF : {(df_fin['Poids']*df_fin['Annualized return 20Y']).sum()}")
print(f"TAXO : {(df_fin['Poids']*df_fin['EST_EU_TAXONOMY_MAX_REV']).sum()}") 
print(f"ITR : {(df_fin['Poids']*df_fin['ITR']).sum()}") 
print(f"Poids : {(df_fin['Poids']).sum()}") 
filtres_durables_social = (df_fin['DURABLE_SOCIAL'] >= 2).astype(int)  # 1 si durable > 2, sinon 0.astype(int)  # 1 si durable > 2, sinon 0
print(f"Investissement durable Social: {(filtres_durables_social*df_fin['Poids']).sum()}") 

filtres_durables_env = (df_fin['DURABLE_ENVIRONNEMENTAL'] >= 2).astype(int)  # 1 si durable > 2, sinon 0.astype(int)  # 1 si durable > 2, sinon 0
print(f"Investissement durable Environnemental: {(filtres_durables_env*df_fin['Poids']).sum()}") 

col_scores = ['E_SCORE', 'S_SCORE', 'G_SCORE']
for col in col_scores:
    print(f"{col} : {(df_fin[col]*df_fin['Poids']).sum()}") 

col_inten = ['INTENSITY_SCOPE_1', 'INTENSITY_SCOPE_2', 'INTENSITY_SCOPE_3']
for col in col_inten:
    print(f"{col} : {(df_fin[col]*df_fin['Poids']).sum()}") 

    
col_foot = ['FOOTPRINT_SCOPE_1', 'FOOTPRINT_SCOPE_2', 'FOOTPRINT_SCOPE_3']
for col in col_foot:
    print(f"{col} : {(df_fin[col]*df_fin['Poids']).sum()}")

for col in odds:
    print(f"{col} : {(df_fin[col]*df_fin['Poids']).sum()}")

Return PTF : 0.28987002516315086
TAXO : 12.00000000000097
ITR : 1.9500000001109505
Poids : 1.0000000000001157
Investissement durable Social: 0.8500000000001147
Investissement durable Environnemental: 0.6698652253264854
E_SCORE : 7.268062690905014
S_SCORE : 5.597863604926251
G_SCORE : 6.407510844872837
INTENSITY_SCOPE_1 : 11.29835516455257
INTENSITY_SCOPE_2 : 11.148004355390022
INTENSITY_SCOPE_3 : 265.6895142491933
FOOTPRINT_SCOPE_1 : 8.363491448048961
FOOTPRINT_SCOPE_2 : 5.168962650713841
FOOTPRINT_SCOPE_3 : 169.51439749457904
SDG_01_NET_ALIGNMENT_SCORE : 0.7919310389543441
SDG_02_NET_ALIGNMENT_SCORE : 0.2000000000000104
SDG_03_NET_ALIGNMENT_SCORE : 0.4121551738847957
SDG_04_NET_ALIGNMENT_SCORE : 0.2583547703981499
SDG_05_NET_ALIGNMENT_SCORE : 2.289865664696557
SDG_06_NET_ALIGNMENT_SCORE : 0.7498625885539615
SDG_07_NET_ALIGNMENT_SCORE : 1.1765025446108008
SDG_08_NET_ALIGNMENT_SCORE : 1.6682142828097524
SDG_09_NET_ALIGNMENT_SCORE : 0.7232204350945537
SDG_10_NET_ALIGNMENT_SCORE : 1.87342

### Resultats MSCI World

In [14]:
print(f"Return PTF : {(df['Weight in MSCI World']*df['Annualized return 20Y']).sum()}")
print(f"TAXO : {(df['Weight in MSCI World']*df['EST_EU_TAXONOMY_MAX_REV']).sum()}") 
print(f"ITR : {(df['Weight in MSCI World']*df['ITR']).sum()}") 
print(f"Poids : {(df['Weight in MSCI World']).sum()}") 


col_scores = ['E_SCORE', 'S_SCORE', 'G_SCORE']
for col in col_scores:
    print(f"{col} : {(df[col]*df['Weight in MSCI World']).sum()}") 

col_inten = ['INTENSITY_SCOPE_1', 'INTENSITY_SCOPE_2', 'INTENSITY_SCOPE_3']
for col in col_inten:
    print(f"{col} : {(df[col]*df['Weight in MSCI World']).sum()}") 

col_foot = ['FOOTPRINT_SCOPE_1', 'FOOTPRINT_SCOPE_2', 'FOOTPRINT_SCOPE_3']
for col in col_foot:
    print(f"{col} : {(df[col]*df['Weight in MSCI World']).sum()}")

for col in odds:
    print(f"{col} : {(df[col]*df['Weight in MSCI World']).sum()}")

Return PTF : 0.1589968660964774
TAXO : 8.88141198983
ITR : 2.3882738824
Poids : 0.999999286
E_SCORE : 6.702299149800001
S_SCORE : 5.1125413661
G_SCORE : 5.6482565943
INTENSITY_SCOPE_1 : 75.69229832018408
INTENSITY_SCOPE_2 : 19.946423978449946
INTENSITY_SCOPE_3 : 756.2246020384931
FOOTPRINT_SCOPE_1 : 118.00648809367438
FOOTPRINT_SCOPE_2 : 27.010985263301677
FOOTPRINT_SCOPE_3 : 1373.604056364562
SDG_01_NET_ALIGNMENT_SCORE : 0.0775825055
SDG_02_NET_ALIGNMENT_SCORE : 0.019335309000000002
SDG_03_NET_ALIGNMENT_SCORE : -0.111505415
SDG_04_NET_ALIGNMENT_SCORE : -0.022077644
SDG_05_NET_ALIGNMENT_SCORE : 1.9017966155000001
SDG_06_NET_ALIGNMENT_SCORE : 0.9266744179999999
SDG_07_NET_ALIGNMENT_SCORE : 0.6206072210000001
SDG_08_NET_ALIGNMENT_SCORE : 1.0800233255
SDG_09_NET_ALIGNMENT_SCORE : 0.100041403
SDG_10_NET_ALIGNMENT_SCORE : 1.0660665655
SDG_11_NET_ALIGNMENT_SCORE : -0.03663600200000001
SDG_12_NET_ALIGNMENT_SCORE : 1.034473784
SDG_13_NET_ALIGNMENT_SCORE : 0.8416237785000001
SDG_14_NET_ALIGNMEN

### Analyse des résultats

In [31]:
# Repartition sectorielle
round((df_fin.groupby('gics_sector_name')['Poids'].sum())*100,2)

gics_sector_name
Communication Services    15.00
Consumer Discretionary    12.67
Consumer Staples           0.00
Energy                     0.00
Financials                 5.00
Health Care               20.34
Industrials                5.00
Information Technology    30.00
Materials                  5.00
Real Estate                6.99
Utilities                  0.00
Name: Poids, dtype: float64

In [32]:
# Sous secteur
df_sect = df_fin.groupby('industry_group')['Poids'].sum()
round(df_sect[df_sect>0.01]*100,2)

industry_group
Apparel                    5.00
Biotechnology              5.34
Commercial Services        5.00
Computers                  5.00
Diversified Finan Serv     5.00
Electronics                5.00
Forest Products&Paper      5.00
Healthcare-Products        5.00
Internet                   6.67
Media                      5.00
Pharmaceuticals            5.00
REITS                      1.99
Real Estate               10.00
Retail                     6.00
Semiconductors            10.00
Software                  15.00
Name: Poids, dtype: float64

In [33]:
# Repartition geo
df_geo = df_fin.groupby('region_en')['Poids'].sum()
round(df_geo[df_geo>0.01]*100,2)

region_en
Asia-Pacific      5.00
Europe           30.34
Latin America     1.67
North America    62.99
Name: Poids, dtype: float64

In [34]:
# Repartition Note
df_esg = df_fin.groupby('IVA_COMPANY_RATING')['Poids'].sum()
round(df_esg[df_esg>0.01]*100,2)

IVA_COMPANY_RATING
A      11.67
AA     36.34
AAA    26.99
BB      5.00
BBB    20.00
Name: Poids, dtype: float64

### Visualisations des résultats

In [19]:
# 📌 Calcul des valeurs pour df6 (portefeuille optimisé)
df6_values = [
    (df_fin['Poids'] * df_fin[metric]).sum() if metric in df_fin.columns else None for metric in odds
]

# 📌 Calcul des valeurs pour df (MSCI World)
df_values = [
    (df['Weight in MSCI World'] * df[metric]).sum() if metric in df.columns else None for metric in odds
]

# 📌 Création du graphique
x = np.arange(len(odds))  # Indices des barres

fig = go.Figure()

# 🔹 Barres pour le portefeuille optimisé (df6)
fig.add_trace(go.Bar(
    x=["ODD_" + x.split("_")[1] for x in odds],
    y=df6_values,
    name="Fonds article 9",
    marker_color='blue'
))

# 🔹 Barres pour MSCI World (df)
fig.add_trace(go.Bar(
    x=["ODD_" + x.split("_")[1] for x in odds],
    y=df_values,
    name="MSCI World",
    marker_color='orange'
))

# 📌 Personnalisation du graphique
fig.update_layout(
    title="Comparaison entre notre fonds article 9 et et MSCI World pour les ODD",
    xaxis_title="ODD",
    yaxis_title="Score moyen",
    barmode='group',
    template='plotly_white',  # Thème professionnel
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="black"
    ),
    legend=dict(
        x=0.02, y=0.98,
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

# 📌 Affichage
fig.show()

In [20]:
scores = ['E_SCORE', 'S_SCORE', 'G_SCORE', 'E_CONTROVERSY', 'S_CONTROVERSY', 'G_CONTROVERSY']

# 📌 Calcul des valeurs pour df6 (portefeuille optimisé)
df6_values = [
    (df_fin['Poids'] * df_fin[metric]).sum() if metric in df_fin.columns else None for metric in scores
]

# 📌 Calcul des valeurs pour df (MSCI World)
df_values = [
    (df['Weight in MSCI World'] * df[metric]).sum() if metric in df.columns else None for metric in scores
]

# 📌 Création du graphique
x = np.arange(len(scores))  # Indices des barres

fig = go.Figure()

# 🔹 Barres pour le portefeuille optimisé (df6)
fig.add_trace(go.Bar(
    x=[x for x in scores],
    y=df6_values,
    name="Fonds article 9",
    marker_color='blue'
))

# 🔹 Barres pour MSCI World (df)
fig.add_trace(go.Bar(
    x=[x for x in scores],
    y=df_values,
    name="MSCI World",
    marker_color='orange'
))

# 📌 Personnalisation du graphique
fig.update_layout(
    title="Comparaison entre notre fonds article 9 et et MSCI World pour les scores E,S & G et les scores de controverses",
    xaxis_title="Pilier",
    yaxis_title="Score moyen",
    barmode='group',
    template='plotly_white',
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="black"
    ),
    legend=dict(
        title="Légende",
        x=0.02, y=0.98,
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

# 📌 Affichage
fig.show()

In [21]:
scores = ['INTENSITY_SCOPE_1', 'INTENSITY_SCOPE_2', 'INTENSITY_SCOPE_3', 'FOOTPRINT_SCOPE_1', 'FOOTPRINT_SCOPE_2', 'FOOTPRINT_SCOPE_3']

# 📌 Calcul des valeurs pour df6 (portefeuille optimisé)
df6_values = [
    (df_fin['Poids'] * df_fin[metric]).sum() if metric in df_fin.columns else None for metric in scores
]

# 📌 Calcul des valeurs pour df (MSCI World)
df_values = [
    (df['Weight in MSCI World'] * df[metric]).sum() if metric in df.columns else None for metric in scores
]

# 📌 Création du graphique
x = np.arange(len(scores))  # Indices des barres

fig = go.Figure()

# 🔹 Barres pour le portefeuille optimisé (df6)
fig.add_trace(go.Bar(
    x=[x for x in scores],
    y=df6_values,
    name="Fonds article 9",
    marker_color='blue'
))

# 🔹 Barres pour MSCI World (df)
fig.add_trace(go.Bar(
    x=[x for x in scores],
    y=df_values,
    name="MSCI World",
    marker_color='orange'
))

# 📌 Personnalisation du graphique
fig.update_layout(
    title="Comparaison entre notre fonds article 9 et et MSCI World pour l'empreinte carbone",
    xaxis_title="Pilier",
    yaxis_title="Score moyen",
    barmode='group',
    template='plotly_white',
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="black"
    ),
    legend=dict(
        title="Légende",
        x=0.02, y=0.98,
        bgcolor='rgba(255,255,255,0.7)',
        bordercolor='black',
        borderwidth=1
    )
)

# 📌 Affichage
fig.show()

## Backtest Benchmark

In [22]:
# Import the data 
# import yfinance as yf
# import pandas as pd
# def get_msci_world_history(ticker="SUSW.L", start="2018-01-01", interval="1wk"):
#     """
#     Récupère l'historique des prix de l'ETF MSCI World depuis Yahoo Finance.
    
#     :param ticker: Symbole de l'actif (par défaut URTH pour l'ETF MSCI World)
#     :param start: Date de début de l'historique (format YYYY-MM-DD)
#     :param interval: Intervalle des données (ex: "1d", "1wk", "1mo")
#     :return: DataFrame avec les données historiques
#     """
#     data = yf.download(ticker, start=start, interval=interval)
#     data["Adj Close Base 100"] = (data["Adj Close"] / data["Adj Close"].iloc[0]) * 100
#     return data
# df = get_msci_world_history().rename(columns={"Adj Close Base 100":"MSCI World"})
# df.head()

# df = df[["MSCI World"]]
# df.head()
# df.to_excel("MSCI_World_histo.xlsx")

In [23]:
df = pd.read_excel("Data/Positions_finales.xlsx")
df.head()


,ticker,Poids
0,NVDA US Equity,0.05
1,AVGO US Equity,0.05
2,MA US Equity,0.05
3,NFLX US Equity,0.05
4,CRM US Equity,0.05


In [24]:
weights = df["Poids"]
df_prices = pd.read_excel("Data/Prices.xlsx")
df_prices.head()
print(df_prices.columns[:])

Index(['Date', 'NVDA', 'AVGO', 'MA', 'NFLX', 'CRM', 'NOVO-B.CO', 'NOW', 'EQIX',
       'MELI', 'CMG', 'REGN', 'CSU.TO', 'FTNT', 'LULU', 'ARGX.BR', 'DECK',
       'AHT.L', 'RGEN', 'SCA-B.ST', 'REA.AX', 'SRT3.DE', 'SAGA-B.ST', 'VIV.PA',
       'MSCI_WORLD'],
      dtype='object')


In [25]:
# Calculer la performance de la stratégie pondérée
df_prices["Strategy"] = sum(df_prices[asset] * weight for asset, weight in zip(df_prices.columns[1:-1], weights))

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_prices["Date"], y=df_prices["Strategy"], mode='lines', name="Fonds Article 9"))
fig.add_trace(go.Scatter(x=df_prices["Date"], y=df_prices["MSCI_WORLD"], mode='lines', name="MSCI World SRI"))
fig.update_layout(title="Backtest du fonds Article 9 vs MSCI World SRI",
                  xaxis_title="Date",
                  yaxis_title="Performance",
                  template="plotly_white")
fig.show()